In [1]:
pip install soundfile --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
#!/usr/bin/env python3

# TIMIT_preparation 
# Mirco Ravanelli 
# Mila - University of Montreal 

# July 2018

# Description: 
# This code prepares TIMIT for the following speaker identification experiments. 
# It removes start and end silences according to the information reported in the *.wrd files and normalizes the amplitude of each sentence.
 
# How to run it:
# python TIMIT_preparation.py $TIMIT_FOLDER $OUTPUT_FOLDER data_lists/TIMIT_all.scp 

# NOTE: This script expects filenames in lowercase (e.g, train/dr1/fcjf0/si1027.wav" rather than "TRAIN/DR1/FCJF0/SI1027.WAV)


import shutil
import os
import soundfile as sf
import numpy as np
import sys
import random

in_folders = 'C:/Users/Hamid/main_audio'
in_folder = 'C:/Users/Hamid/aaa'
out_folder = 'C:/Users/Hamid/SincNet_parkinson/parkinson_out'
npy_folder = 'C:/Users/Hamid/SincNet_parkinson/data_lists/TIMIT_labels.npy'
list_file = 'C:/Users/Hamid/SincNet_parkinson/data_lists/TIMIT_all.scp'
italian_directory = 'C:/Users/Hamid/italian_audio'
train_directory = 'C:/Users/Hamid/italian_audio/TRAIN'
test_directory = 'C:/Users/Hamid/italian_audio/TEST'
test_scp = 'C:/Users/Hamid/SincNet_parkinson/data_lists/TIMIT_test.scp'
train_scp = 'C:/Users/Hamid/SincNet_parkinson/data_lists/TIMIT_train.scp'
target_directory = 'C:/Users/Hamid/italian_voice_parkinson'
test_ratio = 0.25


# Rest of your script goes here...

def ReadList(list_file):
 f=open(list_file,"r")
 lines=f.readlines()
 list_sig=[]
 for x in lines:
    list_sig.append(x.rstrip())
 f.close()
 return list_sig

def copy_folder(in_folder,out_folder):
 if not(os.path.isdir(out_folder)):
  shutil.copytree(in_folder, out_folder, ignore=ig_f)

def ig_f(dir, files):
 return [f for f in files if os.path.isfile(os.path.join(dir, f))]




train_directory = 'C:/Users/Hamid/italian_audio/TRAIN'
test_directory = 'C:/Users/Hamid/italian_audio/TEST'
# Read List file

# Replicate input folder structure to output folder
copy_folder(in_folder,out_folder)

def clear_directory(directory):
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isfile(item_path):
            os.unlink(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)


def split_directories(input_dir, train_dir, test_dir, test_ratio):
    clear_directory(train_dir)
    clear_directory(test_dir)

    directories = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]
    random.shuffle(directories)

    # Calculate the number of directories for the test set
    num_test_directories = int(len(directories) * test_ratio)

    # Split the directories into train and test sets
    test_directories = directories[:num_test_directories]
    train_directories = directories[num_test_directories:]
    copied_directories = set()

    # Copy test directories to the test directory
    for directory_name in test_directories:
        if directory_name not in copied_directories:
            source_dir = os.path.join(input_dir, directory_name)
            dest_dir = os.path.join(test_dir, directory_name)
            shutil.copytree(source_dir, dest_dir)
            
    with open(test_scp, 'w') as scp_file:
        for root, _, files in os.walk(test_directory):
            for file_name in files:
                if file_name.lower().endswith('.wav'):
                    relative_path = os.path.relpath(os.path.join(root, file_name), test_directory)
                    relative_path = relative_path.replace('\\', '/')  # Replace backslashes with forward slashes
                    scp_file.write(f"TEST/{relative_path}\n")
    # Copy train directories to the train directory
    for directory_name in train_directories:
        if directory_name not in copied_directories:
            source_dir = os.path.join(input_dir, directory_name)
            dest_dir = os.path.join(train_dir, directory_name)
            shutil.copytree(source_dir, dest_dir)
            copied_directories.add(directory_name)
               
    with open(train_scp, 'w') as scp_file:
        for root, _, files in os.walk(train_directory):
            for file_name in files:
                if file_name.lower().endswith('.wav'):
                    relative_path = os.path.relpath(os.path.join(root, file_name), train_directory)
                    relative_path = relative_path.replace('\\', '/')  # Replace backslashes with forward slashes
                    scp_file.write(f"TRAIN/{relative_path}\n")
                    
    with open(list_file, 'w') as scp_file:
        for directory_name in directories:
            audio_files = [f for f in os.listdir(os.path.join(input_dir, directory_name)) if f.lower().endswith('.wav')]
            dataset_type = "TRAIN" if directory_name in train_directories else "TEST"
            for audio_file in audio_files:
                scp_file.write(f"{dataset_type}/{directory_name}/{audio_file}\n")
split_directories(in_folders, train_directory, test_directory, test_ratio)

list_sig=ReadList(list_file)
